In [2]:
import pandas as pd
from PIL import Image
import io
import json
import matplotlib.pyplot as plt
import re
import tensorflow as tf

In [3]:
parquet_file = [
    'data/train-00000-of-00004-b4aaeceff1d90ecb.parquet',
    'data/train-00001-of-00004-7dbbe248962764c5.parquet',
    'data/train-00002-of-00004-688fe1305a55e5cc.parquet',
    'data/train-00003-of-00004-2d0cd200555ed7fd.parquet'
    ]

df = pd.concat([pd.read_parquet(f) for f in parquet_file])

In [4]:
df.head()

,image,ground_truth
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""menu"": [{""nm"": ""Nasi Campur Bal..."
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""menu"": [{""nm"": ""SPGTHY BOLOGNAS..."
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""menu"": [{""nm"": ""HAKAU UDANG"", ""..."
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""menu"": [{""nm"": ""Bintang Bremer""..."
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""menu"": {""nm"": ""BASO BIHUN"", ""un..."


In [21]:
json.loads(df['ground_truth'].iloc[4])['valid_line']

[{'words': [{'quad': {'x2': 172,
     'y3': 836,
     'x3': 172,
     'y4': 836,
     'x1': 96,
     'y1': 800,
     'x4': 96,
     'y2': 800},
    'is_key': 0,
    'row_id': 2119099,
    'text': 'BASO'},
   {'quad': {'x2': 272,
     'y3': 834,
     'x3': 272,
     'y4': 834,
     'x1': 180,
     'y1': 798,
     'x4': 180,
     'y2': 798},
    'is_key': 0,
    'row_id': 2119099,
    'text': 'BIHUN'}],
  'category': 'menu.nm',
  'group_id': 3,
  'sub_group_id': 0},
 {'words': [{'quad': {'x2': 444,
     'y3': 830,
     'x3': 444,
     'y4': 830,
     'x1': 424,
     'y1': 796,
     'x4': 424,
     'y2': 796},
    'is_key': 0,
    'row_id': 2119099,
    'text': '1'}],
  'category': 'menu.cnt',
  'group_id': 3,
  'sub_group_id': 0},
 {'words': [{'quad': {'x2': 558,
     'y3': 830,
     'x3': 558,
     'y4': 830,
     'x1': 454,
     'y1': 794,
     'x4': 454,
     'y2': 794},
    'is_key': 0,
    'row_id': 2119099,
    'text': '43.636'}],
  'category': 'menu.unitprice',
  'group_id': 3,
  

In [ ]:
import pandas as pd
import tensorflow as tf

# Data input (contoh Anda)
data = json.loads(df['ground_truth'].iloc[4])['valid_line']

# 1. Mengumpulkan teks dan kategorinya
texts = []
labels = []

for entry in data:
    category = entry['category']
    for word in entry['words']:
        texts.append(word['text'])
        labels.append(category)

# 2. Konversi data menjadi DataFrame untuk eksplorasi lebih mudah
df = pd.DataFrame({'text': texts, 'label': labels})

# 3. Encoding label menjadi angka
label_encoder = {label: idx for idx, label in enumerate(df['label'].unique())}
df['label_encoded'] = df['label'].map(label_encoder)

# 4. Membagi data menjadi fitur dan target
X = df['text'].values
y = df['label_encoded'].values

# 5. Tokenisasi teks menggunakan TensorFlow
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_tokenized)

# 6. Menyiapkan dataset TensorFlow
dataset = tf.data.Dataset.from_tensor_slices((X_padded, y))
dataset = dataset.shuffle(buffer_size=1000).batch(32)


# Info tambahan
print("Jumlah kategori:", len(label_encoder))
print("Contoh data tokenisasi:", X_tokenized[:5])
print("Contoh label:", y[:5])


Jumlah kategori: 9
Contoh data tokenisasi: [[5], [6], [7], [1, 2], [1, 2]]
Contoh label: [0 0 1 2 3]


In [8]:
df.iloc[0]['kotak']

[{'words': [{'quad': {'x2': 244,
     'y3': 390,
     'x3': 244,
     'y4': 390,
     'x1': 232,
     'y1': 372,
     'x4': 232,
     'y2': 372},
    'text': '1'},
   {'quad': {'x2': 270,
     'y3': 390,
     'x3': 270,
     'y4': 390,
     'x1': 256,
     'y1': 374,
     'x4': 256,
     'y2': 374},
    'text': 'x'}]},
 {'words': [{'quad': {'x2': 354,
     'y3': 390,
     'x3': 354,
     'y4': 390,
     'x1': 302,
     'y1': 368,
     'x4': 302,
     'y2': 368},
    'text': 'Nasi'},
   {'quad': {'x2': 440,
     'y3': 391,
     'x3': 439,
     'y4': 388,
     'x1': 364,
     'y1': 365,
     'x4': 363,
     'y2': 368},
    'text': 'Campur'},
   {'quad': {'x2': 497,
     'y3': 385,
     'x3': 499,
     'y4': 388,
     'x1': 446,
     'y1': 365,
     'x4': 448,
     'y2': 362},
    'text': 'Bali'}]},
 {'words': [{'quad': {'x2': 618,
     'y3': 388,
     'x3': 618,
     'y4': 388,
     'x1': 542,
     'y1': 362,
     'x4': 542,
     'y2': 362},
    'text': '75,000'}]},
 {'words': [{'quad': 